In [ ]:
#import libraries for generating report
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.metrics import precision_score, recall_score, f1_score
#Import Pandas
import pandas as pd
#Import numpy
import numpy as np

In [ ]:
#Mount the google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Read the training data from the drive
train = pd.read_csv('/content/drive/MyDrive/real_train.csv',',', names=['Text_data','Label'])


In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
labelencoder = LabelEncoder()
train['Label']=labelencoder.fit_transform(train['Label'])

In [ ]:
print(train)

                                                        Text_data  Label
 PID                                                    Text_data      0
train_pid_1     Waiting for my mind to have a breakdown once t...      1
train_pid_2     My new years resolution : I'm gonna get my ass...      1
train_pid_3     New year : Somone else Feeling like 2020 will ...      1
train_pid_4     My story I guess : Hi, Im from Germany and my ...      1
...                                                           ...    ...
train_pid_8887  Ways to reverse memory loss from depression? :...      3
train_pid_8888  A Comprehensive Guide To Slowly Getting Better...      3
train_pid_8889  I don’t think college is right for me : TW: su...      3
train_pid_8890  Please help: Severe insomnia affecting me in m...      3
train_pid_8891  With each passing day my depression is getting...      3

[8892 rows x 2 columns]


In [ ]:
# Read the development data from the drive
dev = pd.read_csv('/content/drive/MyDrive/real_dev.csv',',', names=['Text_data','Label'])


In [ ]:
print(dev['Label'])

 PID               Label
dev_pid_1       moderate
dev_pid_2       moderate
dev_pid_3       moderate
dev_pid_4       moderate
                  ...   
dev_pid_4492      severe
dev_pid_4493      severe
dev_pid_4494      severe
dev_pid_4495      severe
dev_pid_4496      severe
Name: Label, Length: 4497, dtype: object


In [ ]:
labelencoder = LabelEncoder()
dev['Label']=labelencoder.fit_transform(dev['Label'])
print(dev['Label'])

 PID            0
dev_pid_1       1
dev_pid_2       1
dev_pid_3       1
dev_pid_4       1
               ..
dev_pid_4492    3
dev_pid_4493    3
dev_pid_4494    3
dev_pid_4495    3
dev_pid_4496    3
Name: Label, Length: 4497, dtype: int64


In [ ]:
!pip install simpletransformers

     |████████████████████████████████| 248 kB 5.0 MB/s 
     |████████████████████████████████| 6.8 MB 58.0 MB/s 
     |████████████████████████████████| 43 kB 2.7 MB/s 
     |████████████████████████████████| 1.7 MB 55.6 MB/s 
     |████████████████████████████████| 1.2 MB 56.7 MB/s 
     |████████████████████████████████| 312 kB 63.5 MB/s 
     |████████████████████████████████| 3.5 MB 46.1 MB/s 
     |████████████████████████████████| 9.7 MB 45.9 MB/s 
     |████████████████████████████████| 596 kB 46.8 MB/s 
     |████████████████████████████████| 67 kB 2.8 MB/s 
     |████████████████████████████████| 895 kB 42.5 MB/s 
     |████████████████████████████████| 143 kB 34.1 MB/s 
     |████████████████████████████████| 180 kB 56.8 MB/s 
     |████████████████████████████████| 97 kB 7.2 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
     |████████████████████████████████| 1.1 MB 48.5 MB/s 
     |████████████████████████████████| 133 kB 49.2 MB/s 
     |█████████████████

In [ ]:
from simpletransformers.classification import ClassificationModel

model1=ClassificationModel('albert','albert-base-v1',num_labels=4,use_cuda=True,args={
        "reprocess_input_data" : True,
        "use_cached_eval_features":False, 
        "overwrite_output_dir": True, 
        "num_train_epochs": 3 }) #Increase for better performance

Downloading:   0%|          | 0.00/684 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/45.2M [00:00<?, ?B/s]

Some weights of the model checkpoint at albert-base-v1 were not used when initializing AlbertForSequenceClassification: ['predictions.dense.weight', 'predictions.decoder.weight', 'predictions.decoder.bias', 'predictions.LayerNorm.bias', 'predictions.bias', 'predictions.LayerNorm.weight', 'predictions.dense.bias']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v1 and are newly initialized: ['classifier.weight', 'classifier.bias']
You sho

Downloading:   0%|          | 0.00/742k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25M [00:00<?, ?B/s]

In [ ]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
import re

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.lower()
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    rem_tag = re.sub(r'@\S+', '',rem_num)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_tag)  
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
    return " ".join(filtered_words)

train['Text_data']=train['Text_data'].map(lambda s:preprocess(s))


In [ ]:
dev['Text_data']=dev['Text_data'].map(lambda s:preprocess(s))

In [ ]:
model1.train_model(train)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:586: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/8892 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/1112 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Running Epoch 1 of 3:   0%|          | 0/1112 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/1112 [00:00<?, ?it/s]

(3336, 0.47146439629135656)

In [ ]:
predictions, raw_outputs = model1.predict(dev['Text_data'].tolist())
print(predictions)

  0%|          | 0/4497 [00:00<?, ?it/s]

  0%|          | 0/563 [00:00<?, ?it/s]

[2 2 1 ... 3 2 2]


In [ ]:
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

from sklearn.metrics import classification_report
print(classification_report(dev['Label'], predictions))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.60      0.58      0.59      2306
           2       0.53      0.56      0.55      1830
           3       0.40      0.38      0.39       360

    accuracy                           0.56      4497
   macro avg       0.38      0.38      0.38      4497
weighted avg       0.56      0.56      0.56      4497



In [ ]:
# Read the test data from the drive
test = pd.read_csv('/content/drive/MyDrive/Real_test.csv',',', names=['text data'])

In [ ]:
test['text data']=test['text data'].map(lambda s:preprocess(s))

In [ ]:
test

,text data
Pid,text data
test_pid_1,scared lie every day say ill make think might ...
test_pid_2,new wanted vent finally realized kind bad ment...
test_pid_3,sad kinda always issue say bad peers definitel...
test_pid_4,lonely alone immediately family members dead d...
...,...
test_pid_3241,feeling lonely reddit posted sub hard day hard...
test_pid_3242,would suicide right got back hospital weeks ag...
test_pid_3243,lowest ever ever make long story short made co...
test_pid_3244,toxoplasma gondii ruined life first part gonna...


In [ ]:
predictions1, raw_outputs = model1.predict(test['text data'].tolist())
print(predictions1)

  0%|          | 0/3246 [00:00<?, ?it/s]

  0%|          | 0/406 [00:00<?, ?it/s]

[2 1 1 ... 1 2 3]


In [ ]:
prediction2 = labelencoder.inverse_transform(predictions1)
print(prediction2)

['not depression' 'moderate' 'moderate' ... 'moderate' 'not depression'
 'severe']


In [ ]:
recover= pd.read_csv('/content/drive/MyDrive/Real_test.csv',',', names=['text data'])

In [ ]:
submission = pd.DataFrame()
submission['text data'] = recover['text data']
submission['Label'] = prediction2
submission.to_csv('/content/drive/MyDrive/albert.csv')